# iTerator
## Modular workflow for improved short-read assemblies


In [2]:
import subprocess
import sys
import time
import csv
import shutil
import pandas as pd
import os
import glob

## Setting paths
Set paths in relation to the directory that the notebook is running in.

Paths must be set for SPAdes, QUAST, bbmap, SSPACE/Gapfiller, and Multi-CSAR

(SSPACE and GapFiller directories should be combined)

Paths must also be set for the directories containg reads and references

In [3]:
SPAdes_dir = './SPAdes-3.14.0-Linux'
quast_dir = './quast-5.0.2'
bbmap_dir = './bbmap'
SSPACE_GF_dir = './SSPACE_and_GapFiller'
MultiCSAR_dir =  './Multi-CSAR'

reads_dir = './Mb_reads'
ref_dir = './Mb_refs/'

## Module 1: multi-assembly with SPAdes

Read naming convention: prefix_R1.fastq.gz (forward) and prefix_R2.fastq.gz (reverse)

**read_to_process**: list of read prefixes, indicating which reads you would like assembled.

**best_metric**: metric by which to select best assembly. Current options are any of the columns in the QUAST output, such as 'Largest contig', '# contigs', 'Total length', 'N50', 'N75', 'L50', and 'L75'.

**max_v_min**: determines if best assembly is selected via max or min value of best_metric



In [ ]:
#Best Assemblies (x12)
reads_to_process = ['3A2', '4A3', '4A5', '4G3', 'CL1', 'CL2', 'PNG']
best_metric = 'Longest contig'
max_v_min = 'max'

##############################################################################

print('=== Start! ===')

for x in reads_to_process:
    print('Starting process with '+x+' reads')
    if 'ref' in os.listdir('.'):
        !rm -r ./ref
    if 'temp_assemblies' in os.listdir('.'):
        !rm -r ./temp_assemblies
    !mkdir ./temp_assemblies

    #Error correct reads
    print('read error correction')
    error_correct = SPAdes_dir +'/bin/spades.py -o ./reads_error_correct  --only-error-correction --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(error_correct, shell=True)

    #Assemblies - Standard
    print('assembling reads with 12x parameter sets - this will take a while')
    a1 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a1 --only-assembler --careful -k 21,33,55 --pe1-1 ./reads_error_correct/corrected/'+x+'_R1.fastq.00.0_0.cor.fastq.gz --pe1-2 ./reads_error_correct/corrected/'+x+'_R2.fastq.00.0_0.cor.fastq.gz'
    subprocess.call(a1, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a1'):
        print('a1 assembly failed; moving to next set of reads')
        continue
    
    a2 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a2 --only-assembler --careful -k 21,33,55,77 --pe1-1 ./reads_error_correct/corrected/'+x+'_R1.fastq.00.0_0.cor.fastq.gz --pe1-2 ./reads_error_correct/corrected/'+x+'_R2.fastq.00.0_0.cor.fastq.gz'
    subprocess.call(a2, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a2'):
        print('a2 assembly failed; moving to next set of reads')
        continue
    
    a3 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a3 --only-assembler --careful -k 21,33,55,77,99 --pe1-1 ./reads_error_correct/corrected/'+x+'_R1.fastq.00.0_0.cor.fastq.gz --pe1-2 ./reads_error_correct/corrected/'+x+'_R2.fastq.00.0_0.cor.fastq.gz'
    subprocess.call(a3, shell=True)   
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a3'):
        print('a3 assembly failed; moving to next set of reads')
        continue
    
    a4 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a4 --only-assembler --careful -k 21,33,55,77,99,127 --pe1-1 ./reads_error_correct/corrected/'+x+'_R1.fastq.00.0_0.cor.fastq.gz --pe1-2 ./reads_error_correct/corrected/'+x+'_R2.fastq.00.0_0.cor.fastq.gz'
    subprocess.call(a4, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a4'):
        print('a4 assembly failed; moving to next set of reads')
        continue
    
    #Assemblies - Meta
    a1 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a1m --only-assembler --meta -k 21,33,55 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a1, shell=True)   
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a1m'):
        print('a1m assembly failed; moving to next set of reads')
        continue
    
    a2 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a2m --only-assembler --meta -k 21,33,55,77 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a2, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a2m'):
        print('a2m assembly failed; moving to next set of reads')
        continue
    
    a3 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a3m --only-assembler --meta -k 21,33,55,77,99 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a3, shell=True) 
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a3m'):
        print('a3m assembly failed; moving to next set of reads')
        continue
    
    a4 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a4m --only-assembler --meta -k 21,33,55,77,99,127 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a4, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a4m'):
        print('a4m assembly failed; moving to next set of reads')
        continue
    
    #Assemblies - Isolate
    a1 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a1i --only-assembler --isolate -k 21,33,55 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a1, shell=True)  
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a1i'):
        print('a1i assembly failed; moving to next set of reads')
        continue
    
    a2 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a2i --only-assembler --isolate -k 21,33,55,77 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a2, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a2i'):
        print('a2i assembly failed; moving to next set of reads')
        continue
    
    a3 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a3i --only-assembler --isolate -k 21,33,55,77,99 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a3, shell=True)  
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a3i'):
        print('a3i assembly failed; moving to next set of reads')
        continue
    
    a4 = SPAdes_dir + '/bin/spades.py -o ./temp_assemblies/a4i --only-assembler --isolate -k 21,33,55,77,99,127 --pe1-1 ' + reads_dir + '/'+x+'_R1.fastq.gz --pe1-2 ' + reads_dir + '/'+x+'_R2.fastq.gz'
    subprocess.call(a4, shell=True)
    if 'contigs.fasta' not in os.listdir('./temp_assemblies/a4i'):
        print('a4i assembly failed; moving to next set of reads')
        continue

    #Annotate all assemblies with Prokka (may need to change contig names)
    print('annotating all assemblies with prokka')
    anno = ''
    for y in ['a1','a2','a3','a4','a1m','a2m','a3m','a4m','a1i','a2i','a3i','a4i']:
        anno = anno + 'prokka --outdir ./temp_assemblies/prokka_'+y+' --centre X --compliant --cpus 0 --prefix '+y+' ./temp_assemblies/'+y+'/contigs.fasta ; '
    subprocess.call(anno, shell=True)
    
    #Count genes and compare all assemblies
    print('counting ORFs in all assemblies')
    count = ''
    for y in ['a1','a2','a3','a4','a1m','a2m','a3m','a4m','a1i','a2i','a3i','a4i']:
        count = count + './count_cds_partials.pl ./temp_assemblies/prokka_'+y+'/'+y+'.ffn >> ./temp_assemblies/counts.txt ; '
    subprocess.call(count, shell=True)
    
    #Check lengths, number of contigs
    print('checking assembly lengths, number of contigs')
    quast = quast_dir + '/quast.py -o ./temp_assemblies/quast_output ./temp_assemblies/a*/contigs.fasta'
    subprocess.call(quast, shell=True)
    
    #Check coverage of a2
    print('checking coverage to determine if suitable for submitting to celera')
    f = open("./temp_assemblies/coverage.txt", "w")
    cov = bbmap_dir + '/bbmap.sh in1=./reads_error_correct/corrected/'+x+'_R1.fastq.00.0_0.cor.fastq.gz in2=./reads_error_correct/corrected/'+x+'_R2.fastq.00.0_0.cor.fastq.gz ref=./temp_assemblies/a2/contigs.fasta covstats=./temp_assemblies/covstats.txt'
    subprocess.call(cov, shell=True, stderr=f)
    
    #Compile coverage data, so it is all in one place
    print('adding coverage data to masterfile')
    t = open("./temp_assemblies/coverage.txt", "r")
    for line in t:
        if 'Average coverage:' in line:
            av_cov = line
            av_cov = float(av_cov.split('\t')[1].split('\n')[0])
            print('average coverage for '+x+' is '+str(av_cov))
    with open("./assembly_database/assemblies_cov.txt", "a+") as file:
        file.seek(0)
        data = file.read(100)
        if len(data) > 0 :
            file.write("\n")
        file.write('average coverage for '+x+' is '+str(av_cov))

    #Move all data generated to assembly_database, to document
    print('compiling results')
    mkdir_x = 'mkdir ./assembly_database/'+x+'_data'
    subprocess.call(mkdir_x, shell=True)
    cp_cov = 'cp ./temp_assemblies/coverage.txt ./assembly_database/'+x+'_data/'+x+'_coverage.txt'
    subprocess.call(cp_cov, shell=True)
    cp_counts = 'cp ./temp_assemblies/counts.txt ./assembly_database/'+x+'_data/'+x+'_orf-counts.txt'
    subprocess.call(cp_counts, shell=True)
    cp_quast = 'cp ./temp_assemblies/quast_output/report.tsv ./assembly_database/'+x+'_data/'+x+'_quast-report.tsv'
    subprocess.call(cp_quast, shell=True)
    
    #Move best assembly to 'best assembly' dir
    print('preserving assembly with the longest contig')
    quast = pd.read_csv('./assembly_database/'+x+'_data/'+x+'_quast-report.tsv', sep='\t', header=0)
    quast.index = quast['Assembly']
    quast.drop('Assembly', axis=1, inplace=True)
    if max_v_min == 'max':
        ba = quast.T[best_metric].idxmax()
    elif max_v_min == 'min':
        ba = quast.T[best_metric].idxmin()
    ba = ba.split('_')[0]
    best_assembly = './temp_assemblies/'+ba+'/contigs.fasta'
    mv_script = 'cp '+best_assembly+' ./best_assemblies/'+x+'_'+ba+'_contigs.fasta'
    subprocess.call(mv_script, shell=True)
    
    #Delete rest of assemblies
    print('cleaning up before next round')
    !rm -r ./ref
    !rm -r ./temp_assemblies
    !rm -r ./reads_error_correct

print('=== Complete! ===')



## Module 2: iterative scaffolding and gapfilling, for improved contiguity

Prior to starting module 2, all assemblies need library files, stored in the SSPACE/GapFiller dir (see SSPACE and/or GapFiller documentation)

The expected naming convention for library files is libraries_**SamplePrefix**_F_updatedIS.txt

**assemblies_to_iT**: list of assembly prefixes to improve

In [ ]:
assemblies_to_iT = ['4G3']

###############################################################################################

for sample in assemblies_to_iT:
    print('Now starting with '+ sample)
    refs_to_record =  glob.glob(ref_dir + '*')
    print('Refs:')
    print(refs_to_record)
    ref_contigs =  glob.glob(ref_dir + '*' + sample + '_*')
    if ref_contigs == 1:
        print('There is already a reference genome for this sample code. Let\'s see if we can improve on it...')
        clean_refs = 'rm ' + ref_dir + ref_contigs[0].split('/')[-1]
        print(clean_refs)
        subprocess.call(clean_refs, shell=True)
        contig_path = './iT_assemblies/' + ref_contigs[0].split('/')[-1]
        print('contigs: ' + contig_path)    
    else:
        print('No reference genomes yet for this sample code.')
        assemblies_path = "./best_assemblies/"
        sample_contigs = glob.glob(assemblies_path+'*' + sample + '_*')
        if len(sample_contigs) == 1:
            contig_path = sample_contigs[0]
            print('contigs: ' + contig_path)
        else:
            print('Could not find contigs for the sample')
            continue
        
    prefix = sample
    print('prefix: ' + prefix)
    library_list = glob.glob(SSPACE_GF_dir + '/*_' + sample + '_F_updatedIS.txt')
    if len(library_list) == 1:
        library = '.' + library_list[0]
        print('library_path: ' + library)
    else:
        print('Could not find library for the sample')
        continue
        
    prog_doc = sample + '_iT_prog.csv'
    print('name of progress doc: ' + prog_doc)
    


    with open(prog_doc, 'wb') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',')

    it_contigs = []

    process_start_time = time.time()

    for i in range(0,20):
        print('Start of Iteration '+str(i))
        print('==========================================================')
        iteration_start_time = time.time()
        if i == 0:
            print('Counting contigs, to see where we are starting')
            with open(contig_path,'r') as f:

                file = [l.replace('\n','') for l in f]

            scaffs=0
            for line in file:
                if line.startswith(">"):
                    scaffs += 1
            print('Starting with '+str(scaffs)+' contigs')
            fields=[i,scaffs]
            with open(prog_doc, 'a') as f:
                writer = csv.writer(f)
                writer.writerow(['sample', sample])
                writer.writerow(['refs', refs_to_record])
                writer.writerow(['contigs', contig_path])
                writer.writerow(['library_path', library])
                writer.writerow(fields)
            
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            continue
        print('Iteration '+str(i)+': SSPACE starting')
        T0 = 'cd '+ SSPACE_GF_dir +' ; '
        if i == 1:
            SS = './SSPACE_Standard_v3.0.pl -l '+library+' -s .'+contig_path+' -x 1 -o 10 -r 0.75 -b '+prefix+'_sspace_'+str(i)+' -T 12 ; '
            print('using original contigs')
        else:
            SS = './SSPACE_Standard_v3.0.pl -l '+library+' -s ../alt_fasta.fasta -x 1 -o 10 -r 0.75 -b '+prefix+'_sspace_'+str(i)+' -T 12 ; '
            print('using alt_fasta')
        run_1 = T0+SS
        print(run_1)
        subprocess.call(run_1, shell=True)
        print('Iteration '+str(i)+': SSPACE completed')
        print('iteration time elapsed = '+ str(round((time.time()-iteration_start_time)/60,2))+ ' mins')
        print('process time elapsed = '+ str(round((time.time()-process_start_time)/60/60,2))+ ' hours')
        print('==========================================================')
        
        print('Iteration '+str(i)+': Multi-CSAR starting')
        MC = MultiCSAR_dir + '/multi-csar.php -t SSPACE_and_GapFiller/'+prefix+'_sspace_'+str(i)+'/'+prefix+'_sspace_'+str(i)+'.final.scaffolds.fasta -r Mb_refs --nuc -o '+prefix+'_multi-csar_'+str(i)+' ;'
        run_2 = MC
        print(run_2)
        subprocess.call(run_2, shell=True)
        print('Iteration '+str(i)+': Multi_CSAR completed')
        print('iteration time elapsed = '+ str(round((time.time()-iteration_start_time)/60,2))+ ' mins')
        print('process time elapsed = '+ str(round((time.time()-process_start_time)/60/60,2))+ ' hours')
        print('==========================================================')
    
        shutil.rmtree(SSPACE_GF_dir + '/'+prefix+'_sspace_'+str(i), ignore_errors=True)
        print('*Removed SSPACE file, keeping things tidy!!')
        print('==========================================================')
        
        print('Iteration '+str(i)+': GapFiller starting')
        T2 = 'cd SSPACE_and_GapFiller ; '
        GF = './GapFiller.pl -l '+library+' -s ../'+prefix+'_multi-csar_'+str(i)+'/multi-csar.nuc.out.fna -b '+prefix+'_gapfilled_'+str(i)+' -T 12 -i 1000000 ;'
        run_3 = T2+GF
        print(run_3)
        subprocess.call(run_3, shell=True)
        print('Iteration '+str(i)+': GapFiller completed')
        print('iteration time elapsed = '+ str(round((time.time()-iteration_start_time)/60,2))+ ' mins')
        print('process time elapsed = '+ str(round((time.time()-process_start_time)/60/60,2))+ ' hours')
        print('==========================================================')
    
        shutil.rmtree('./'+prefix+'_multi-csar_'+str(i), ignore_errors=True)
        print('*Removed Multi-CSAR file, keeping things tidy!!')
        print('==========================================================')

        print('Iteration '+str(i)+': ScaffoldChopper starting')
        alt_fasta=[]
        with open('SSPACE_and_GapFiller/'+prefix+'_gapfilled_'+str(i)+'/'+prefix+'_gapfilled_'+str(i)+'.gapfilled.final.fa','r') as f:

            file = [l.replace('\n','') for l in f]

        seq=[]
        for line in file:
            if line.startswith(">"):
                alt_fasta.append(line)
                last_scaf = line
                scaf_num = 0
            elif 'N' in line:
                if line.count('N') > 0:
                    new_scafs = line.replace('N'," ").split()
                    for x in range(len(new_scafs)):
                        if scaf_num == 0:
                            scaf_num = scaf_num + 1
                            alt_fasta.append(new_scafs[x])
                        else:
                            alt_fasta.append(last_scaf+'-'+str(scaf_num))
                            scaf_num = scaf_num + 1
                            alt_fasta.append(new_scafs[x])
                    
                    
                else:
                    alt_fasta.append(line)
            else:
                alt_fasta.append(line)
        
        
    
        with open('alt_fasta.fasta', 'w') as filehandle:
            filehandle.writelines("%s\n" % line for line in alt_fasta)
        

        with open('alt_fasta.fasta','r') as f:

            file = [l.replace('\n','') for l in f]

        scaffs=0
        for line in file:
            if line.startswith(">"):
                scaffs += 1
            
            
        print('Iteration '+str(i)+': Gaps removed to make all scaffolds contiguous')
        if i == 1:
            save1 = 'cp ./alt_fasta.fasta ./iT_assemblies/iT_' + sample + '_' + str(i) + '_' + str(scaffs) + '.fasta'
            subprocess.call(save1, shell=True)
            print('Iteration 1 saved for future reference')
        print('Iteration '+str(i)+' done!')
        print('iteration time elapsed = '+ str(round((time.time()-iteration_start_time)/60,2))+ ' mins')
        print('process time elapsed = '+ str(round((time.time()-process_start_time)/60/60,2))+ ' hours')
        print('==========================================================')
    
        shutil.rmtree(SSPACE_GF_dir + '/'+prefix+'_gapfilled_'+str(i), ignore_errors=True)
        print('*Removed GapFiller file, keeping things tidy!!')
        print('==========================================================')
    
        print('There are still '+str(scaffs)+' contigs after '+str(i)+' iterations')
        it_contigs.append([i,scaffs])
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
       
        fields=[i,scaffs]
        with open(prog_doc, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
        
        if i > 15:    
            df=pd.read_csv(prog_doc, header=None)
            if (int(df[1].iloc[-11]) - int(df[1].iloc[-1]))/10 < 1:
                break
            else:
                print('Still decreasing the number of contigs')
    print('Just finished. Contigs were no longer decreasing.')
    save = 'mv ./alt_fasta.fasta ./iT_assemblies/iT_' + sample + '_' + str(i) + '_' + str(scaffs) + '.fasta'
    subprocess.call(save, shell=True)
    move_prog = 'mv ./' + prog_doc + ' ./iT_progs/' + sample + '_' + str(i) + '_' + str(scaffs) + '_iT_prog.csv'
    subprocess.call(move_prog, shell=True)
    #to_refs = 'cp ./iT_assemblies/iT_' + sample + '_' + str(i) + '_' + str(scaffs) + '.fasta ./Mb_refs/iT_' + sample + '_' + str(i) + '_' + str(scaffs) + '.fasta'
    #subprocess.call(to_refs, shell=True)
    


## Integration with Celera/WGS, for better assemblies with sufficient coverage
### Still under development

In [ ]:
#Submit eligible reads to celera/WGS
reads_to_process = ['3A2']

for x in reads_to_process:
    print('Starting celera process with '+x+' reads')

    #Error correct reads
    print('read error correction')
    error_correct = './SPAdes-3.14.0-Linux/bin/spades.py -o ./reads_error_correct-'+x+'  --only-error-correction --pe1-1 ./Mb_reads/'+x+'_R1.fastq.gz --pe1-2 ./Mb_reads/'+x+'_R2.fastq.gz'
    subprocess.call(error_correct, shell=True)
    
    



In [ ]:
!./wgs-8.3rc2/Linux-amd64/bin/fastqToCA -insertsize 215 76 -libraryname 3A2_rec_215_76 -technology illumina-long -type sanger -mates ./reads_error_correct-3A2/corrected/3A2_R1.fastq.00.0_0.cor.fastq.gz,./reads_error_correct-3A2/corrected/3A2_R2.fastq.00.0_0.cor.fastq.gz > 3A2_rec_215_76.frg

In [ ]:
!./wgs-8.3rc2/Linux-amd64/bin/runCA -d ./3A2_0rec -p 3A2_0rec -s ./3A2-0rec.spec.txt


In [ ]:
!./quast-5.0.2/quast.py ./3A2-1F/9-terminator/3A2-1F.ctg.fasta ./3A2_0rec/9-terminator/3A2_0rec.ctg.fasta ./best_assemblies/3A2_a3_contigs.fasta
